<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/Sovits_f0%E4%B8%80%E9%94%AE%E5%90%88%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

**适用于此脚本关联的sovits2.0模型**

**[旧模型合成](https://colab.research.google.com/drive/14GT8uWM9IATjVylYNGGVXiKWHaDHZSTf)**

sovits包括**训练、合成**两部分，替换至本篇进行合成的模型必须是**Rcell版引入f0参数的sovits方式训练出的模型（三件套的colab，模型仅在内部互通）**

**格式参考vits专栏三件套（评论区）**[vits注解](https://www.bilibili.com/read/cv18478187)

95%的问题都可以参考专栏解决，剩下的我也不会了

[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n?usp=drive_open#scrollTo=xx2oAf90btEy)

[一键训练](https://colab.research.google.com/drive/1DexYpwWIdD_RRqQ165l-YoWMzFAHIbPy?usp=drive_open)

**支持一键合成长时间的音频（5min以上），建议使用GPU（CPU比较慢）**

hubert.pt为[soft-vc](https://github.com/bshall/hubert)发布的内容合成器模型，G.pth为R佬在huggingface发布的模型；采用存在谷歌云盘的方式，节约下载时间。
[Sovits_f0版本](https://github.com/IceKyrin/sovits_f0_infer) fork自rcell佬的[github](https://github.com/innnky/so-vits-svc)，其中内置了R佬pth的config.json及官方hubert模块（改为加载本地模型方式），以方便使用。

# 配置环境

In [1]:
!git clone -b record https://github.com/IceKyrin/sovits_f0_infer
%cd sovits_f0_infer
!pip install -r requirements.txt
!mkdir pth
!mkdir raw
!mkdir results
%cd wav_temp
!mkdir input
!mkdir output
%cd ..

Cloning into 'sovits_f0_infer'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 274 (delta 138), reused 194 (delta 87), pack-reused 0
Receiving objects: 100% (274/274), 100.36 KiB | 14.34 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/content/sovits_f0_infer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 183 kB 34.5 MB/s 
     |████████████████████████████████| 11.6 MB 55.9 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 25.9 MB 1.9 MB/s 
     |████████████████████████████████| 238 kB 56.9 MB/s 
     |████████████████████████████████| 212 kB 51.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 131 kB 56.2 MB/

In [2]:
import logging
import os
import shutil
!pip install demjson pydub pyworld
import demjson
import soundfile
import torch
import torchaudio
import hubert_model
import infer_tool
import utils
from models import SynthesizerTrn
from preprocess_wave import FeatureInput
from wav_temp import merge



def get_units(path):
    source, sr = torchaudio.load(path)
    source = torchaudio.functional.resample(source, sr, 16000)
    source = source.unsqueeze(0).to(dev)
    with torch.inference_mode():
        units = hubert_soft.units(source)
        return units


def transcribe(path, length, transform):
    feature_pit = featureInput.compute_f0(path)
    feature_pit = feature_pit * 2 ** (transform / 12)
    feature_pit = infer_tool.resize2d_f0(feature_pit, length)
    coarse_pit = featureInput.coarse_f0(feature_pit)
    return coarse_pit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached demjson-2.2.4-py3-none-any.whl
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl


# 加载模型

## 加载内容编码器

In [3]:
# 这个东西是https://github.com/bshall/hubert/releases/tag/v0.1 的hubert-soft-0d54a1f4.pt，可以自己替换来源、但是不能换其他模型（路径自己改）。
#!gdown --id '1cA37nsiSnsouF2TJkaXb3_VoA-rbifTu' --output /content/sovits_f0_infer/pth/hubert.pt
!wget https://huggingface.co/spaces/xiaolang/sovits_midi_dev/resolve/main/hubert.pt -O /content/sovits_f0_infer/pth/hubert.pt
hubert_soft = hubert_model.hubert_soft('/content/sovits_f0_infer/pth/hubert.pt')

--2022-10-03 06:14:54--  https://huggingface.co/spaces/xiaolang/sovits_midi_dev/resolve/main/hubert.pt
Resolving huggingface.co (huggingface.co)... 54.173.5.192, 54.210.225.113, 52.5.54.249, ...
Connecting to huggingface.co (huggingface.co)|54.173.5.192|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/28/d8/28d830556bcdd0cb46c4fede42ac3941296568acdd6086982580ca95e028d6ca/e82e7d079df05fe3aa535f6f7d42d309bdae1d2a53324e2b2386c56721f4f649?response-content-disposition=attachment%3B%20filename%3D%22hubert.pt%22 [following]
--2022-10-03 06:14:55--  https://cdn-lfs.huggingface.co/repos/28/d8/28d830556bcdd0cb46c4fede42ac3941296568acdd6086982580ca95e028d6ca/e82e7d079df05fe3aa535f6f7d42d309bdae1d2a53324e2b2386c56721f4f649?response-content-disposition=attachment%3B%20filename%3D%22hubert.pt%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.227.254.123, 13.227.254.47, 13.227.254.52, ...
Connecting to cdn-lfs.hugging

## 加载生成器

如果要**替换自己的模型**，将 !gdown这行注释掉（行首加个“#”即可，注释成功则变绿）

将**自己的配置json（上一篇生成了的）**上传至/content/sovits_infer_rcell/configs/文件夹
将**自己的模型（上一篇生成了的）**上传至/content/sovits_infer_rcell/pth文件夹

In [4]:
from google.colab import drive

#@markdown 是否使用谷歌盘内模型（不勾选则自动下载猫雷模型）
g_drive = True #@param {type:"boolean"}
if g_drive:
  drive.mount('/content/drive/')
  config_path = "/content/drive/MyDrive/lamianer/config.json" #@param {type:"string"}
  model_path = '/content/drive/MyDrive/lamianer/G_21000.pth' #@param {type:"string"}
else:
  # 这个东西是https://huggingface.co/spaces/innnky/soft-vits-singingvc 的G.pth（猫雷），可以换成自己的模型（必须是按照sovits方式训练出的其他角色模型）
  !wget https://huggingface.co/spaces/innnky/nyaru-svc2.0/resolve/main/nyarumodel.pth -O /content/sovits_f0_infer/pth/G.pth  
  # !gdown --id '1c8ao-0leUPKg6pb8wnF43nSPIzjAHiGE' --output /content/sovits_f0_infer/pth/G.pth
  config_path = "./configs/nyarumul.json"
  model_path = "/content/sovits_f0_infer/pth/G.pth"

hps_ms = utils.get_hparams_from_file(config_path)
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net_g_ms = SynthesizerTrn(
    178,
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
_ = utils.load_checkpoint(model_path, net_g_ms, None)
_ = net_g_ms.eval().to(dev)
targer_sample_rate = hps_ms.data.sampling_rate
featureInput = FeatureInput(hps_ms.data.sampling_rate, hps_ms.data.hop_length)

Mounted at /content/drive/


# 声音转换

支持{1、2}**任选一个方式**的声音转换！
支持**10s以上5分钟以内**的音频（再久合成时间会过长）。
上传到/content/sovits_infer_rcell/raw文件夹，支持自动合成歌曲

使用[spleeter](https://github.com/deezer/spleeter)的2stems模式分离歌曲，自动生成这两个文件。（请自行阅读官方使用文档）

spleeter separate -p spleeter:2stems -o output audio_example.mp3

**结果自动输出至results文件夹。**自行下载，无预览
mp3为自动合成的带伴奏歌曲，out_vits为纯人声。

跑调破音基本是因为直播采样到的音域不够，这个没办法。（狗头）猫雷高音上不去、低音下不去。
例子是牵丝戏，可以不下。明显感觉开头的低音、戏腔都炸了，其他部分还好。

In [ ]:
# 进results下载试听
!gdown --id '1ymJDK1VSESzv2xv_2Ce8h4QoSnzoplt7' --output /content/sovits_f0_infer/results/demo.mp3

1、使用参考音频

In [ ]:
!gdown --id '10JQMPdzp0gjg9cVVersxVZWhIr4UwrFF' --output /content/sovits_f0_infer/raw/vocals.wav
!gdown --id '1lwmw9P-kgNYjjUveD8J_HuF_yfcdnoHJ' --output /content/sovits_f0_infer/raw/bgm.wav

2、使用上传音频

自行上传至raw文件夹（单声道，22050hz，wav格式），可有bgm.wav（必须为wav格式），无伴奏则为纯人声合成

3、合成音频

In [5]:
#@markdown **仅支持wav格式**

#@markdown 角色id——猫雷模型：0号为猫雷，1号为？？？

#@markdown 角色id
speaker_id = "0" #@param {type:"string"}

#@markdown 人声文件名（不带.wav）
clean_name = "summer_ED2_vocal" #@param {type:"string"}
#@markdown 伴奏文件名（可以不放伴奏）（不带.wav）
bgm_name = "" #@param {type:"string"}
#@markdown 每次处理的长度，建议30s以内，大了炸显存
cut_time = "15" #@param {type:"string"}
#@markdown 可为正负（升降n个半音）
vc_transform = "0" #@param {type:"string"}


out_audio_name = clean_name
# 可填写音源文件列表，音源文件格式为wav，放置于raw文件夹下
clean_names = [clean_name]
# bgm、trans分别对应歌曲列表，若能找到相应文件、则自动合并伴奏，若找不到bgm，则输出干声（不使用bgm合成多首歌时，可只随意填写一个不存在的bgm名）
bgm_names = [bgm_name]
# 合成多少歌曲时，若半音数量不足、自动补齐相同数量（按第一首歌的半音）
trans = [int(vc_transform)]  # 加减半音数（可为正负）s
# 每首歌同时输出的speaker_id
id_list = [int(speaker_id)]

# 每次合成长度，建议30s内，太高了爆掉显存(gtx1066一次15s以内）
cut_time = int(cut_time)

# 自动补齐
infer_tool.fill_a_to_b(bgm_names, clean_names)
infer_tool.fill_a_to_b(trans, clean_names)
for clean_name, bgm_name, tran in zip(clean_names, bgm_names, trans):
    infer_tool.resample_to_22050(f'./raw/{clean_name}.wav')
    for speaker_id in id_list:
        speakers = ["猫雷","？？？"]
        out_audio_name = clean_name
        # 清除缓存文件
        infer_tool.del_file("./wav_temp/input/")
        infer_tool.del_file("./wav_temp/output/")

        raw_audio_path = f"./raw/{clean_name}.wav"
        audio, sample_rate = torchaudio.load(raw_audio_path)

        audio_time = audio.shape[-1] / targer_sample_rate
        if audio_time > 1.3 * int(cut_time):
            infer_tool.cut(int(cut_time), raw_audio_path, out_audio_name, "./wav_temp/input")
        else:
            shutil.copy(f"./raw/{clean_name}.wav", f"./wav_temp/input/{out_audio_name}-0.wav")
        file_list = os.listdir("./wav_temp/input")

        count = 0
        for file_name in file_list:
            source_path = "./wav_temp/input/" + file_name
            audio, sample_rate = torchaudio.load(source_path)
            input_size = audio.shape[-1]

            sid = torch.LongTensor([int(speaker_id)]).to(dev)
            soft = get_units(source_path).squeeze(0).cpu().numpy()
            pitch = transcribe(source_path, soft.shape[0], tran)
            pitch = torch.LongTensor(pitch).unsqueeze(0).to(dev)
            stn_tst = torch.FloatTensor(soft)

            with torch.no_grad():
                x_tst = stn_tst.unsqueeze(0).to(dev)
                x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(dev)
                audio = \
                    net_g_ms.infer(x_tst, x_tst_lengths, pitch, sid=sid, noise_scale=.667, noise_scale_w=0.8,
                                   length_scale=1)[0][
                        0, 0].data.float().cpu().numpy()

            soundfile.write("./wav_temp/output/" + file_name, audio, int(audio.shape[0] / input_size * targer_sample_rate))
            count += 1
            print("%s success: %.2f%%" % (file_name, 100 * count / len(file_list)))
        merge.run(out_audio_name, bgm_name, out_audio_name)


summer_ED2_vocal-1.wav success: 5.00%
summer_ED2_vocal-16.wav success: 10.00%
summer_ED2_vocal-7.wav success: 15.00%
summer_ED2_vocal-3.wav success: 20.00%
summer_ED2_vocal-14.wav success: 25.00%
summer_ED2_vocal-13.wav success: 30.00%
summer_ED2_vocal-5.wav success: 35.00%
summer_ED2_vocal-12.wav success: 40.00%
summer_ED2_vocal-4.wav success: 45.00%
summer_ED2_vocal-19.wav success: 50.00%
summer_ED2_vocal-15.wav success: 55.00%
summer_ED2_vocal-9.wav success: 60.00%
summer_ED2_vocal-2.wav success: 65.00%
summer_ED2_vocal-8.wav success: 70.00%
summer_ED2_vocal-18.wav success: 75.00%
summer_ED2_vocal-0.wav success: 80.00%
summer_ED2_vocal-10.wav success: 85.00%
summer_ED2_vocal-6.wav success: 90.00%
summer_ED2_vocal-11.wav success: 95.00%
summer_ED2_vocal-17.wav success: 100.00%
out vits success


In [ ]:
import IPython.display as ipd
import torchaudio
#@markdown 预览干声（自行进results查看文件名，带.wav，仅支持wav）
source_path = "/content/sovits_f0_infer/results/vocals.wav"  #@param {type:"string"}
audio,sr = torchaudio.load(source_path)
ipd.display(ipd.Audio(audio, rate=sr))